<a href="https://colab.research.google.com/github/taeyang1224/Final_Project/blob/develop/2.Model_code/DeepFM_%ED%83%9C%EC%96%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


# DeepFM

In [ ]:
!pip install -U deepctr-torch

     |████████████████████████████████| 70 kB 3.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

# 파일 불러오기

- 이미지 벡터 없는 파일

In [ ]:
df = pd.read_csv('data/total_data.csv')
# vec_df = pd.read_csv('data/vec512.csv', low_memory=False)
df.head(2)

,content_id,imp_id,view_time,bid_price,user_id,label,imp_time,adv_id,content_name,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,content_delivery_fee,content_status,content_img_url,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count,viewer_age_ch,content_place,b_pay_rate,sec,title_len
0,137078818,99d1612dded122458e99,16:56:46,50,1728109,1,16:48:41,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,배송비포함,삭제됨,https://media.bunjang.co.kr/product/137078818_...,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,30,서울특별시,0.394224,484.0,34
1,137078818,99d1612d825321055790,00:00:00,50,10733469,1,17:00:45,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,배송비포함,삭제됨,https://media.bunjang.co.kr/product/137078818_...,2131,5285,247,4525,542,739,496,0,0,11,1,18,0,0,0,0,0,18,서울특별시,0.394224,7.0,34


# 준비

- 평가함수 정의

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score

grouped_label = df.groupby('label').size()
average_ctr = float(grouped_label[1]/grouped_label.sum())
average_ctr

def get_rig(train_y, test_y, pred):
    avg_ctr = average_ctr
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig


# 전처리

### 학습에 사용할 feature 분류

In [ ]:
df["viewer_age_ch10"] = df["viewer_age_ch"].apply(lambda x : (x//10) * 10)

In [ ]:
categorical = [
    'viewer_gender',  
    'content_used',
    'content_cat_1',
    #'content_cat_2',
    #'content_cat_3',
    "content_status",
    'content_delivery_fee',
    'content_b_pay', 
    'content_place',
    "viewer_age_ch10"
    ]

continuous = [
    'bid_price', 
    'content_price',
    #'content_emergency_count',
    'content_comment_count',     
    'content_views',       
    'content_likes',
    'adv_follower_count',
    #'adv_grade',       
    'adv_item_count',         
    'adv_views',
    'adv_review_count',
    'adv_comment_count',
    'adv_pay_count',
    'adv_parcel_post_count', 
    'adv_transfer_count', 
    #'adv_chat_count',
    # 'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    "viewer_parcel_post_count",
    'viewer_transfer_count',
    'viewer_chat_count', 
#     'b_pay_rate',
    # 'viewer_age_ch',
    #'sec', 
    'title_len'
    ]

# 이미지 벡터 사용하는 경우
# vec_features = [str(i) for i in range(512)]
# continuous = continuous + vec_features

### 결측치

In [ ]:
# 결측치 채우기
df[categorical] = df[categorical].fillna('-1', )
df[continuous] = df[continuous].fillna(0, )
target = ['label']

### Label Encoding, Scaling

In [ ]:
# categorical 데이터는 Label Encoding
# for feat in categorical:
#     lbe = LabelEncoder()
#     df[feat] = lbe.fit_transform(df[feat])
    
# continuous 데이터는 Min-Max Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df[continuous] = scaler.fit_transform(df[continuous])

# # continuous 데이터는 Standard Scaling
# scaler = StandardScaler()
# df[continuous] = scaler.fit_transform(df[continuous])

# scaler = RobustScaler()
# df[continuous] = scaler.fit_transform(df[continuous])

In [ ]:
df = pd.get_dummies(df, columns=categorical, drop_first=True)

In [ ]:
categorical = df.columns[37:].tolist()

In [ ]:
df.columns[37:]

Index(['viewer_gender_1', 'viewer_gender_2', 'content_used_1',
       'content_cat_1_가방', 'content_cat_1_기타', 'content_cat_1_남성 악세사리',
       'content_cat_1_남성 의류', 'content_cat_1_도서/티켓/문구', 'content_cat_1_미용품',
       'content_cat_1_반려동물용품', 'content_cat_1_생활/가공식품', 'content_cat_1_스타굿즈',
       'content_cat_1_스포츠/레저', 'content_cat_1_시계/쥬얼리', 'content_cat_1_신발',
       'content_cat_1_여성 의류', 'content_cat_1_예술/희귀/수집품',
       'content_cat_1_유아동/출산', 'content_cat_1_음반/악기', 'content_cat_1_전자제품',
       'content_cat_1_지역 서비스', 'content_cat_1_차량/오토바이', 'content_cat_1_키덜트',
       'content_status_삭제됨', 'content_status_판매중',
       'content_delivery_fee_배송비 별도', 'content_delivery_fee_배송비포함',
       'content_b_pay_1', 'content_place_경상남도', 'content_place_경상북도',
       'content_place_대구광역시', 'content_place_대전광역시', 'content_place_부산광역시 ',
       'content_place_서울특별시', 'content_place_인천광역시', 'content_place_잠실역 (서',
       'content_place_전국', 'content_place_전라북도 전', 'content_place_충청북도',
       'v

### train test split

In [ ]:
# 3.generate input data for model

train, test = train_test_split(df, test_size=0.2, random_state=47)#, stratify=df['label'])

### 각 feature 정보를 형식에 맞춰 저장

In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name

# 각 feature마다 값의 종류가 몇개인지, feature 이름이 무엇인지 등 저장
fixlen_feature_columns = [SparseFeat(feat, df[feat].nunique())
                            for feat in categorical] + [DenseFeat(feat, 1, )
                                                            for feat in continuous]

# dnn feature에 일단 모든 feature 정보 넣기
dnn_feature_columns = fixlen_feature_columns

# lenear feature에도 일단 모든 feature 정보 넣기
linear_feature_columns = fixlen_feature_columns 

# feature 이름들만 따로 저장
feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

In [ ]:
# train/test 데이터의 각 feature에 대응하는 컬럼을 딕셔너리로 저장
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

# 학습

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary')


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=1, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cpu
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch


920it [00:39, 23.58it/s]


Epoch 1/20
41s - loss:  0.1389 - binary_crossentropy:  0.1389 - auc:  0.6785 - val_binary_crossentropy:  0.1322 - val_auc:  0.6909


920it [00:38, 24.04it/s]


Epoch 2/20
40s - loss:  0.1365 - binary_crossentropy:  0.1365 - auc:  0.6788 - val_binary_crossentropy:  0.1330 - val_auc:  0.6825


...

Epoch 18/20
41s - loss:  0.1293 - binary_crossentropy:  0.1293 - auc:  0.7431 - val_binary_crossentropy:  0.1303 - val_auc:  0.7140


920it [00:39, 23.57it/s]


Epoch 19/20
41s - loss:  0.1291 - binary_crossentropy:  0.1290 - auc:  0.7457 - val_binary_crossentropy:  0.1316 - val_auc:  0.7140


920it [00:38, 23.79it/s]


Epoch 20/20
41s - loss:  0.1288 - binary_crossentropy:  0.1288 - auc:  0.7490 - val_binary_crossentropy:  0.1300 - val_auc:  0.7191

test LogLoss 0.1329
test AUC 0.7176
test RIG 0.0613


- DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(256, 128), l2_reg_linear=1e-05, l2_reg_embedding=1e-05, l2_reg_dnn=0, init_std=0.0001, seed=1024, dnn_dropout=0, dnn_activation='relu', dnn_use_bn=False, task='binary', device='cpu', gpus=None)
### Parameters:	
- linear_feature_columns – An iterable containing all the features used by linear part of the model.
- dnn_feature_columns – An iterable containing all the features used by deep part of the model.
- use_fm – bool,use FM part or not
- dnn_hidden_units – list,list of positive integer or empty list, the layer - - number and units in each layer of DNN
- l2_reg_linear – float. L2 regularizer strength applied to linear part
- l2_reg_embedding – float. L2 regularizer strength applied to embedding vector
- l2_reg_dnn – float. L2 regularizer strength applied to DNN
- init_std – float,to use as the initialize std of embedding vector
- seed – integer ,to use as random seed.
- dnn_dropout – float in [0,1), the probability we will drop out a given DNN coordinate.
- dnn_activation – Activation function to use in DNN
- dnn_use_bn – bool. Whether use BatchNormalization before activation or not in DNN
- task – str, "binary" for binary logloss or "regression" for regression loss
- device – str, "cpu" or "cuda:0"
- gpus – list of int or torch.device for multiple gpus. If None, run on device. gpus[0] should be the same gpu with device.

# 학습 - 1223

- Standard Scaling, 이미지 벡터 제외

In [ ]:
# 4.Define Model,train,predict and evaluate
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

# optimizer, loss 설정
model.compile("adagrad", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )
# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=1024, epochs=20, verbose=2, validation_split=0.0)

# 예측
pred_ans = model.predict(test_model_input, 1024)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 588555 samples, validate on 0 samples, 575 steps per epoch
Epoch 1/20
8s - loss:  0.1388 - binary_crossentropy:  0.1388 - auc:  0.6745
Epoch 2/20
8s - loss:  0.1350 - binary_crossentropy:  0.1350 - auc:  0.6879
Epoch 3/20
8s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6896
Epoch 4/20
8s - loss:  0.1346 - binary_crossentropy:  0.1346 - auc:  0.6899
Epoch 5/20
8s - loss:  0.1346 - binary_crossentropy:  0.1346 - auc:  0.6918
Epoch 6/20
8s - loss:  0.1345 - binary_crossentropy:  0.1344 - auc:  0.6924
Epoch 7/20
8s - loss:  0.1344 - binary_crossentropy:  0.1344 - auc:  0.6931
Epoch 8/20
8s - loss:  0.1343 - binary_crossentropy:  0.1343 - auc:  0.6942
Epoch 9/20
8s - loss:  0.1342 - binary_crossentropy:  0.1342 - auc:  0.6941
Epoch 10/20
8s - loss:  0.1342 - binary_crossentropy:  0.1342 - auc:  0.6956
Epoch 11/20
8s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6955
Epoch 12/20
8s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6

- Standard Scaling, 이미지 벡터 전부 사용

In [ ]:
# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=1024, epochs=20, verbose=2, validation_split=0.0)

# 예측
pred_ans = model.predict(test_model_input, 1024)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda:0
Train on 588555 samples, validate on 0 samples, 575 steps per epoch
Epoch 1/20
11s - loss:  0.1303 - binary_crossentropy:  0.1303 - auc:  0.7360
Epoch 2/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7356
Epoch 3/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7365
Epoch 4/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7366
Epoch 5/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7366
Epoch 6/20
11s - loss:  0.1301 - binary_crossentropy:  0.1301 - auc:  0.7376
Epoch 7/20
11s - loss:  0.1301 - binary_crossentropy:  0.1301 - auc:  0.7374
Epoch 8/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7381
Epoch 9/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7387
Epoch 10/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7386
Epoch 11/20
11s - loss:  0.1299 - binary_crossentropy:  0.1299 - auc:  0.7391
Epoch 12/20
11s - loss:  0.1299 - binary_crossentropy:  0.1299 - auc:  0.739